# Convolutional Neural Network - MNIST

In this notebook, my goal is to implement a CNN to classify handwritten digits from the MNIST dataset using PyTorch. 

Classifying handwritten digits is not new to me at this point 😅. In the previous folder, I used PyTorch to implement an MLP to do the very same thing, and acheived 96% accuracy on the testing set, which I think is pretty good! 🤷

So, implementing this is CNN is also a chance for me to compare the performance of two different architectures (CNN and MLP) on the very same task. In `README.md` of this folder, I introduced CNNs as a family of models that are *well-suited for image-related task*. By the end of this notebook, I will see for myself! The score to beat is: 96%

Okay, let's jump right in!

# Model Overview

The architecture of the network that I am going to implement is shown on the following image:

![Deep CNN architecture](./images/img-7.jpg)



# Loading and Preprocessing the data

# Implementing the model using PyTorch

# Training the model

# Last words...